# Node Label Prediction \ Link Prediction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import networkx as nx
%matplotlib inline

We will start by node label prediction. Download [this](https://www.dropbox.com/s/sta1krsbnnp7ju0/ppi.CC.gml?dl=0) network. It contains protein communications in Baker’s yeast. Each node (protein) has a special binary attribute *ICSC (intracellular signaling cascade)*.

In [ ]:
g = nx.read_gml('ppi.CC.gml')
cc = list(nx.connected_components(g))
g = nx.subgraph(g,cc[0])
g = nx.relabel.convert_node_labels_to_integers(g)

In [ ]:
labels = np.array(nx.get_node_attributes(g, 'ICSC').values(), dtype=float)

In [ ]:
nx.draw_spring(g, node_color = labels)

It might not be clear from the picture above but the level of homogeneity is quite high. For each node we are able to compute the average value of label

In [ ]:
nnICSC = np.asarray(map(lambda(v): np.mean(labels[g.neighbors(v)]), g.nodes_iter()))
nnICSC
plt.figure(figsize=(10,5))
plt.hist(nnICSC[np.where(labels == 1)], bins=6,)

## Iterative Classification Method

ICM is kind of NN-classifier. Our prediction is based on the largest ratio of nearest neighbours of unlabeled nodes.

#### Task 1

* Randomly set unlabeled nodes. 
* Implement classification rule of ICM (HINT look at the code cell above)
* Implement function for classification error and use it wisely

In [ ]:
lablNN = labels[:]
idx = np.random.randint(0,len(lablNN), size=40)
lablNN[idx] = np.nan

In [ ]:
# Your code here


## Label Propagation

Now instead of looking at neigbours we are switching random walks between all the nodes

Just to recall the Label Propagation method:
1. Compute $P = D^{-1}A$
2. Set $Y^{(0)} = (Y_l,0)$ ($Y_l$ - labeled data)
3. **repeat**
    * $Y^{(t+1)} = PY^{(t)}$
    * $Y_l^{(t+1)} = Y_l$
4. **until** $Y^{(t)}$ converges

In [ ]:
# It is better to initialize like that
fixedLabels = labels[:]+1
curLabels = labels[:]+1

# And indicate labeled nodes instead of unlabeled
idxLabeled = np.zeros((g.number_of_nodes(),), dtype=bool)
idxLabeled[np.random.randint(0,len(labels), size=90)] = True
curLabels[~idxLabeled] = 0

In [ ]:
def LabelPropagation(G, idxLabeled, curLabels, fixedLabels, iters = 1000):
    A = np.asarray(nx.adj_matrix(G).todense())
    P = np.diag(1.0/sum(A)).dot(A)
    
    # Your code here
    return np.round(resultLabels)

## Link Prediction - Scoring functions

In this section we will implement some scoring functions for link prediction and compare the values for adjacent and non-adjacent nodes.

Load [french blog network](https://www.dropbox.com/s/rn0y18a511vfx1t/fblog.gml?dl=0) and compute the following scores:

In [ ]:
g = nx.read_gml('fblog.gml')
vNum = g.number_of_nodes()

In [ ]:
def matrixPlot(A):
    plt.figure(1, figsize=(6, 6))
    plt.imshow(A,
           interpolation="none"
           )

#### Shortest Path Length

In [ ]:
# Your code here


#### Number of common neighbours

In [ ]:
# Your code here


#### Jaccard Score

In [ ]:
# Your code here


#### Adamic/Adar Score

$Score(a,b) = \sum\limits_{v \in \text{NN}(a) \cap \text{NN}(b)} \frac{1}{|\text{NN}(v)|}$

In [ ]:
# Your code here


#### Preferential Attachment score

$Score(a,b) = |\text{NN}(a)| \times |\text{NN}(b)|$

In [ ]:
# Your code here


#### Katz Score

$Score(a,b) = \sum\limits_{\text{Paths}_{x,y}} \beta^{|p_{a,b}|}\times|p_{a,b}|$

In [ ]:
# Your code here


Let's compare the scores behavious for pairs of nodes with or without edge in-between

In [ ]:
A = np.asarray(nx.adj_matrix(g).todense())
xyTriu = np.vstack(np.triu_indices_from(A, k=1)).T
wEdge = [katzScore[xy[0],xy[1]] for xy in xyTriu if A[xy[0],xy[1]]]
woEdge = [katzScore[xy[0],xy[1]] for xy in xyTriu if ~A[xy[0],xy[1]]]
data = [wEdge, woEdge]

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
axes.violinplot(data, showmeans=True)
axes.set_xticklabels(['', 'With Edges', '', 'W/o Edges'])